In [81]:
UC = [0] + [5]*9 + [25]*7 + [5]*8
CAPA=[15,14,16,10,17,16,15,18]
inventory = [30,28,32,20,34,32,30,36]
SC = 100
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
#각 시간별 각 기계의 생산량
P={}
for i in range(8):
    for j in range(1,25):
        P[i,j]=solver.IntVar(0, solver.infinity(), 'p[%i,%i]' % (i,j))
#각 시간별 각 기계의 가동 여부
Y={}
for i in range(8):
    for j in range(25):
        Y[i,j]=solver.BoolVar('x[%i,%i]' % (i,j))
#각 시간 말의 각 기계별 재고량
I={}
for i in range(8):
    for j in range(25):
        I[i,j]=solver.IntVar(0, solver.infinity(), 'i[%i,%i]' % (i,j))
#setup시간 발생 여부
S={}
for i in range(8):
    for j in range(1,25):
        S[i,j]=solver.BoolVar('s[%i,%i]' % (i,j))



In [82]:
#제약조건1: 각 시간별 생산량은 앞서 쌓인 재고량보다 작거나 같다
for i in range(1,8):
    for j in range(1,25):
        solver.Add(P[i,j] <= I[i,j])
#첫 기계는 재고에 의한 생산량 제약 없음
#제약조건2: 초기 재고량은 주어진 값과 같다.
for i in range(7):
    solver.Add(I[i,0] == inventory[i])
solver.Add(I[7,0] == 0)  # 마지막 기계의 초기 재고량은 0으로 설정
#제약조건3: 각 재고량은 생산량과 이전 재고량의 합으로 계산된다.
for i in range(7):
    for j in range(1,25):
        solver.Add(I[i,j] == I[i,j-1] + P[i,j]- P[i+1,j])
#제약조건4: 마지막 재고량은 초기 재고량과 같아야한다.
for i in range(7):
    solver.Add(I[i,24] == inventory[i])
for j in range(1,25):
    solver.Add(I[7,j] == I[7,j-1] + P[7,j])  # 마지막 기계의 재고량은 이전 시간의 재고량과 현재 생산량의 합
solver.Add(I[7,24] == min(CAPA)*24)  # 마지막 기계의 최종 재고량은 병목공정의 생산량량으로 설정

#제약조건5: 각 기계의 생산량은 해당 기계의 가동 여부에 따라 결정된다.
for i in range(8):
    for j in range(1,25):
        solver.Add(P[i,j] <= Y[i,j] * CAPA[i])
        solver.Add(P[i,j] >= Y[i,j])#생산이 없는 경우 Y[i,j]는 0이 되게 함
#제약조건6: setup 시간은 가동 여부에 따라 결정된다.
for i in range(1,8):
    for j in range(1,25):
        solver.Add(S[i,j]>=Y[i,j]-Y[i-1,j])

for i in range(8):
    solver.Add(S[i,1] == Y[i,1])  # 첫 번째 기계는 항상 가동되므로 setup 시간은 항상 1
    solver.Add(Y[i,0]==0)


In [83]:
objective_terms = []
for i in range(8):
    for j in range(1, 25):
        objective_terms.append(UC[j] * P[i,j] + SC * S[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print('Total Cost =', solver.Objective().Value())
    print('Solution:')
    for i in range(8):
        for j in range(1, 25):
            print(f'Machine {i}, Hour {j}: Production = {P[i,j].solution_value()}, Setup = {S[i,j].solution_value()}, Inventory = {I[i,j].solution_value()}')



Total Cost = 11760.0
Solution:
Machine 0, Hour 1: Production = 0.0, Setup = 0.0, Inventory = 21.0
Machine 0, Hour 2: Production = 15.0, Setup = 0.0, Inventory = 22.0
Machine 0, Hour 3: Production = 15.0, Setup = 0.0, Inventory = 23.0
Machine 0, Hour 4: Production = 15.0, Setup = 0.0, Inventory = 24.0
Machine 0, Hour 5: Production = 15.0, Setup = 0.0, Inventory = 25.0
Machine 0, Hour 6: Production = 15.0, Setup = 0.0, Inventory = 26.0
Machine 0, Hour 7: Production = 15.0, Setup = 0.0, Inventory = 27.0
Machine 0, Hour 8: Production = 15.0, Setup = 0.0, Inventory = 28.0
Machine 0, Hour 9: Production = 15.0, Setup = 0.0, Inventory = 29.0
Machine 0, Hour 10: Production = 1.0, Setup = 0.0, Inventory = 29.0
Machine 0, Hour 11: Production = 1.0, Setup = 0.0, Inventory = 29.0
Machine 0, Hour 12: Production = 1.0, Setup = 0.0, Inventory = 29.0
Machine 0, Hour 13: Production = 1.0, Setup = 0.0, Inventory = 29.0
Machine 0, Hour 14: Production = 1.0, Setup = 0.0, Inventory = 29.0
Machine 0, Hour 15

| 표현            | 의미                   |
| ------------- | -------------------- |
| `df.shape`    | `(행 수, 열 수)`를 튜플로 반환 |
| `df.shape[0]` | 행(row)의 수            |
| `df.shape[1]` | 열(column)의 수         |



In [84]:
import pandas as pd
import numpy as np
#판다스가 엑셀의 데이터를 불러올 때 자동으로 Unnamed: 0 이라는 열을 생성함.
#index_col='Unnamed: 0'라는 조건을 붙여 Unnamed: 0 열을 다시 인덱스로 돌아가게 해 보이지 않게 함 
#또는 index_col=0을 써도 됨, 'Unnamed: 0'은 그냥 기본값 이름임임
df = pd.read_excel(r"C:\Users\paint\OneDrive\Desktop\ex1.xlsx", index_col='Unnamed: 0')
cList=sorted(set(df['주문번호']))
pList=sorted(set(df['상품코드']))

nClients=len(cList)
nProducts=len(pList)

#0으로 채워진 DataFrame을 생성
#행은 주문번호 리스트, 열은 상품코드 리스트
CP_MAT=pd.DataFrame(0,index=cList,columns=pList,dtype=int)

#loc는 label기반으로 데이터에 접근하는 방법
#df.loc[행이름,열이름]: 지정한 행과 열의 교차점에 있는 데이터를 가져오거나 수정할 수 있음 
for i in range(df.shape[0]): #df.shpe[0]은 pandas dataframe의 행 개수를 의미
    CP_MAT.loc[df['주문번호'][i],df['상품코드'][i]]=df['수량'][i]
CP_MAT_01 =(CP_MAT != 0)*1
print(CP_MAT_01)

      1  2  3  4  5
1001  1  0  1  1  0
1002  1  1  1  0  1
1003  0  0  1  1  0
1004  0  1  1  1  0
1005  1  1  0  1  1
1006  1  0  0  1  0
1007  1  0  0  0  1


주어진 두 사용자 또는 주문에 대한 유사성을 계산   

 ① 내적 (Dot Product)   
 v1*v2 = sum(v1[i]*v2[i] for i in range(n))   
: 같은 인덱스 위치에 있는 두 리스트의 값을 곱하고 그 결과값들을 더함   

 ② 벡터의 크기 (Norm)   
 |v1|=(sum(v1[i]*2 for i in range(n)))**(1/2)   
 : 각 원소를 제곱한 후 더하고 마지막에 루트 씌움 (벡터의 길이 또는 크기)

In [85]:
#Similarity 계산 함수 
def calSimilarity(v1,v2):
    prod,tsum1,tsum2= 0.0,0.0,0.0
    #제곱해준 후 루트를 씌워 절댓값으로 만들어준다 
    for i in range(len(v1)):
        tsum1+=v1[i]*v1[i]
        tsum2+=v2[i]*v2[i]
        prod+=v1[i]*v2[i]
    #벡터의 내적값/벡터의 크기
    return prod/(tsum1**(1/2)*tsum2**(1/2))

#0으로 채워진 DataFrame 생성성
SIMs = pd.DataFrame(0,index=cList,columns=cList, dtype=float)

#Similarity Matrix 값 넣기 
for i in range(SIMs.shape[0]):
    for j in range(SIMs.shape[0]):
        if i<j:
            #iloc=integer-location based indexing, iloc[행번호, 열번호], iloc[행번호]: 선택된 행 전체
            sim_val=calSimilarity(list(CP_MAT_01.iloc[i]),list(CP_MAT_01.iloc[j]))
            SIMs.iloc[i,j]= sim_val
            SIMs.iloc[j,i]= sim_val
print(SIMs)

          1001      1002      1003      1004      1005      1006      1007
1001  0.000000  0.577350  0.816497  0.666667  0.577350  0.816497  0.408248
1002  0.577350  0.000000  0.353553  0.577350  0.750000  0.353553  0.707107
1003  0.816497  0.353553  0.000000  0.816497  0.353553  0.500000  0.000000
1004  0.666667  0.577350  0.816497  0.000000  0.577350  0.408248  0.000000
1005  0.577350  0.750000  0.353553  0.577350  0.000000  0.707107  0.707107
1006  0.816497  0.353553  0.500000  0.408248  0.707107  0.000000  0.500000
1007  0.408248  0.707107  0.000000  0.000000  0.707107  0.500000  0.000000


가장 유사한 구매자 찾기   
| 코드                      | 결과                          |
| ----------------------- | --------------------------- |
| `SIMs.iloc[i]`          | i번째 행 (Series, index는 열 이름) |
| `SIMs.iloc[i].index`    | 열 이름 리스트 (`columns`)        |
| `SIMs.iloc[i].idxmax()` | 가장 유사한 **열의 이름(라벨)**        |
| `SIMs.index[i]`         | i번째 행의 **이름(라벨)**           |

SIMS.index[i] 를 하면 i번째 행의 이름을 반환하지만 SIMs.iloc[i].index는 특정 행을 뽑아낸 상태로 Series 형태이다.   
열에 인덱스가 붙기 때문에 열의 이름을 가져온다 

In [86]:
similarone=[]
for i in range(nClients):
    max_val=max(SIMs.iloc[i])
    #list(SIMs.iloc[i])는 해당 행의 데이터만 포함
    c_user_id=list(SIMs.iloc[i]).index(max_val) #해당 행의 데이터 중 max값을 가지는 열의 인덱스를 불러옴(정수값)
    similarone.append(c_user_id)
    print('user%s and %s are similar'%(SIMs.index[i],SIMs.index[c_user_id]))

user1001 and 1003 are similar
user1002 and 1005 are similar
user1003 and 1001 are similar
user1004 and 1003 are similar
user1005 and 1002 are similar
user1006 and 1001 are similar
user1007 and 1002 are similar


다른 방식   
.idxmax() 사용

In [87]:
for i in range(nClients):
    c_user_id=SIMs.iloc[i].idxmax()
    print('user%s and %s are similar'%(SIMs.index[i],c_user_id))

user1001 and 1003 are similar
user1002 and 1005 are similar
user1003 and 1001 are similar
user1004 and 1003 are similar
user1005 and 1002 are similar
user1006 and 1001 are similar
user1007 and 1002 are similar


상품 추천 함수 작성 

In [89]:
for i in range(nClients):
    idx=similarone[i]
    for j in range(nProducts):
        if CP_MAT_01.iloc[i,j]==False and CP_MAT_01.iloc[idx,j]==True:
            print('We recommend item %s for user %s'%(CP_MAT.columns[j],CP_MAT.index[i]))

We recommend item 4 for user 1002
We recommend item 1 for user 1003
We recommend item 3 for user 1005
We recommend item 3 for user 1006
We recommend item 2 for user 1007
We recommend item 3 for user 1007
